In [1]:
import numpy as np
import pandas as pd

In [2]:
import os

housing_path = os.path.join("..", "datasets", "housing", "housing.csv")
housing_data = pd.read_csv(housing_path)

In [3]:
housing_data["income_cat"] = pd.cut(
    housing_data["median_income"],
    bins=[0.0, 1.5, 3.0, 4.5, 6.0, np.inf],
    labels=[1, 2, 3, 4, 5],
)

In [4]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(
    housing_data, test_size=0.2, random_state=42, stratify=housing_data["income_cat"]
)

for set in (train_set, test_set):
    set.drop("income_cat", axis=1, inplace=True)

In [5]:
X_train = train_set.drop("median_house_value", axis=1)
y_train = train_set["median_house_value"]

X_test = test_set.drop("median_house_value", axis=1)
y_test = test_set["median_house_value"]

In [6]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.preprocessing import OneHotEncoder


class ClusterSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters, gamma, random_state) -> None:
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.random_state = random_state

    def fit(self, X, y=None, sample_weight=None):
        self.kmeans_ = KMeans(self.n_clusters, random_state=self.random_state)
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self  # always return self

    def transform(self, X):
        return rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)

    def get_feature_names_out(self, name=None):
        return [f"Cluster {i} similarity" for i in range(self.n_clusters)]


def column_ratio(X):
    return X[:, [0]] / X[:, [1]]


def ratio_name(function_transformer, feature_names_in):
    return ["ratio"]  # feature names out


def ratio_pipeline():
    return make_pipeline(
        SimpleImputer(strategy="median"),
        FunctionTransformer(column_ratio, feature_names_out=ratio_name),
        StandardScaler(),
    )


cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"), OneHotEncoder(handle_unknown="ignore")
)

log_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log, feature_names_out="one-to-one"),
    StandardScaler(),
)

cluster_simil = ClusterSimilarity(n_clusters=10, gamma=1.0, random_state=42)

default_num_pipeline = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())

preprocessing = ColumnTransformer(
    [
        ("bedrooms", ratio_pipeline(), ["total_bedrooms", "total_rooms"]),
        ("rooms_per_house", ratio_pipeline(), ["total_rooms", "households"]),
        ("people_per_house", ratio_pipeline(), ["population", "households"]),
        (
            "log",
            log_pipeline,
            [
                "total_bedrooms",
                "total_rooms",
                "population",
                "households",
                "median_income",
            ],
        ),
        ("geo", cluster_simil, ["latitude", "longitude"]),
        ("cat", cat_pipeline, make_column_selector(dtype_include=object)),
    ],
    remainder=default_num_pipeline,
)  # one column remaining: housing_median_age

We will implement the features selector by our best model so far, the RandomForestRegressor.

In [7]:
# rnd_search.best_params_ from housing notebook
svr_C = 157055.10989448498
svr_gamma = 0.26497040005002437
svr_kernel = "rbf"

In [8]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

selector_pipeline = Pipeline(
    [
        ("preprocessing", preprocessing),
        (
            "selector",
            SelectFromModel(RandomForestRegressor(random_state=42), threshold=0.005),
        ),
        ("svr", SVR(C=svr_C, gamma=svr_gamma, kernel=svr_kernel)),
    ]
)

In [16]:
from sklearn.model_selection import GridSearchCV

param_grid = {"selector__threshold": [1e-3, 2e-3, 4e-3, 5e-3, 8e-3, 1e-2]}
grid_search = GridSearchCV(
    selector_pipeline, param_grid=param_grid, scoring="neg_mean_squared_error", cv=5
)
grid_search.fit(X_train[:5000], y_train[:5000])

c:\Users\tanda\anaconda3\envs\handons-DS\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\tanda\anaconda3\envs\handons-DS\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\tanda\anaconda3\envs\handons-DS\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\tanda\anaconda3\envs\handons-DS\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                                                     SimpleImputer(strategy='median')),
                                                                                    ('standardscaler',
                                                                                     StandardScaler())]),
                                                          transformers=[('bedrooms',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('functiontransformer',
                                                                                          FunctionTransformer(feature_names_out=<f...
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001DE6ABC10D0>)])),
                                       ('selector',
                                        SelectFromModel(estimator=RandomForestRegressor(random_state=42),
                                                        threshold=0.005)),
                                       ('svr',
                                        SVR(C=157055.10989448498,
                                            gamma=0.26497040005002437))]),
             param_grid={'selector__threshold': [0.001, 0.002, 0.004, 0.005,
                                                 0.008, 0.01]},
             scoring='neg_mean_squared_error')

In [17]:
grid_search.best_params_

{'selector__threshold': 0.004}

In [18]:
grid_search.best_score_

-3059500625.3133345